# IA - Projeto 01: Horário de turma 

# 1. Introdução
### Membros do grupo
- Hugo Ferreira Baptista — nº 23279
- Nuno da Cunha Faria Gajo — nº 23002


### Contexto e Objetivo do projeto
O objetivo deste projeto é para desenvolver um **agente inteligente** capaz de gerar **horários de turmas** que respeitem certas restrições tais como, disponibilidade de professores, salas e evitar conflitos de horários.  
O problema é formulado como um **CSP** utilizando Python e a biblioteca `python-constraint`.


In [ ]:
# Install contraint library
%pip install python-constraint

In [ ]:
# Import contraint library
from constraint import *

# 2. Design do Agente

Nesta seção, definimos o problema do horário de turmas como um **CSP**.

Cada variável representa uma **aula** e pode assumir um valor (bloco de tempo, sala, online) onde:
- `bloco de tempo` \ `block` ∈ [1; 20] (20 blocos por semana);
- `sala` \ `room` ∈ {S1, S2, S3, Lab01};
- `online` ∈ {Verdadeiro, Falso}.

Nós definimos:
- Variáveis e domínios;  
- Restrições rígidas (obrigatórias);
- Restrições flexíveis (opcionais, tratadas posteriormente);
- Heurísticas para melhorar o desempenho;

In [ ]:
# Exemplo de leitura do ficheiro e preparação dos dados:
with open('ClassTT_01_tiny.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())
        

In [ ]:
# =====================================================
#   TIMETABLE CSP - SOLUÇÃO FINAL (AJUSTADA)
# =====================================================

from constraint import Problem
from collections import defaultdict

# --- Recursos ---
rooms = ['R1', 'R2', 'R3', 'Lab01']

classes = {
    't01': ['UC11','UC12','UC13','UC14','UC15'],
    't02': ['UC21','UC22','UC23','UC24','UC25'],
    't03': ['UC31','UC32','UC33','UC34','UC35']
}

teachers = {
    'jo': ['UC11','UC21','UC22','UC31'],
    'mike': ['UC12','UC23','UC32'],
    'rob': ['UC13','UC14','UC24','UC33'],
    'sue': ['UC15','UC25','UC34','UC35']
}

teacher_restrictions = {
    'mike': list(range(13,21)),
    'rob': list(range(1,5)),
    'sue': [9,10,11,12,17,18,19,20]
}

fixed_rooms = {'UC14': 'Lab01', 'UC22': 'Lab01'}

# Apenas estas duas UCs podem ser online
online_allowed = ['UC21', 'UC31']

# --- Inicializar problema ---
problem = Problem()

# Variáveis: (slot, sala, online)
for turma, ucs in classes.items():
    for uc in ucs:
        for i in [1, 2]:
            var = f"{uc}_{i}"
            if uc in online_allowed:
                domain = [(slot, room, online)
                          for slot in range(1, 21)
                          for room in rooms
                          for online in [False, True]]
            else:
                domain = [(slot, room, False)
                          for slot in range(1, 21)
                          for room in rooms]
            problem.addVariable(var, domain)

# =====================================================
#                  HARD CONSTRAINTS
# =====================================================

# --- 1. No máximo 3 aulas por dia por turma ---
def max3_por_dia(*values):
    dias = defaultdict(int)
    for v in values:
        dia = (v[0]-1)//4
        dias[dia] += 1
        if dias[dia] > 3:
            return False
    return True

for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    problem.addConstraint(max3_por_dia, vars_turma)

# --- 2. Professores respeitam disponibilidade ---
for prof, ucs_prof in teachers.items():
    unavailable = teacher_restrictions.get(prof, [])
    for uc in ucs_prof:
        for i in [1,2]:
            problem.addConstraint(lambda v, u=unavailable: v[0] not in u, (f"{uc}_{i}",))

# --- 3. Nenhuma sala usada ao mesmo tempo ---
def no_room_conflict(a, b):
    return not (a[0] == b[0] and a[1] == b[1])

all_vars = [f"{uc}_{i}" for turma in classes.values() for uc in turma for i in [1,2]]
for i in range(len(all_vars)):
    for j in range(i+1, len(all_vars)):
        problem.addConstraint(no_room_conflict, (all_vars[i], all_vars[j]))

# --- 4. Nenhuma turma com 2 aulas ao mesmo tempo ---
def no_turma_conflict(a, b):
    return a[0] != b[0]

for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    for i in range(len(vars_turma)):
        for j in range(i+1, len(vars_turma)):
            problem.addConstraint(no_turma_conflict, (vars_turma[i], vars_turma[j]))

# --- 5. Aulas da mesma UC em dias diferentes ---
def diff_days(a, b):
    return (a[0]-1)//4 != (b[0]-1)//4

for turma, ucs in classes.items():
    for uc in ucs:
        problem.addConstraint(diff_days, (f"{uc}_1", f"{uc}_2"))

# --- 6. Salas fixas ---
for uc, sala in fixed_rooms.items():
    for i in [1,2]:
        problem.addConstraint(lambda v, s=sala: v[1] == s, (f"{uc}_{i}",))

# --- 7. (REMOVIDO) UC21_2 e UC31_2 online no mesmo dia ---
# 🔸 Esta restrição foi eliminada porque agora as aulas online não precisam coincidir.

# --- 8. Cada turma tem pelo menos uma aula na segunda (dia 1) ---
def at_least_one_day1(*values):
    return any((v[0]-1)//4 == 0 for v in values)

for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    problem.addConstraint(at_least_one_day1, vars_turma)

# --- 9. Professores não podem dar aulas ao mesmo tempo ---

#por fazer isto, precisamos de um mapeamento inverso de professores para UCs

# =====================================================
#                  SOLUÇÃO
# =====================================================

print("⏳ Gerando solução...")
solution = problem.getSolution()

if not solution:
    print("❌ Nenhuma solução encontrada.")
else:
    print("\n✅ Solução encontrada!\n")

    horas = ["09-11", "11-13", "14-16", "16-18"]

    # Organizar resultados por turma
    for turma, ucs in classes.items():
        print(f"\n==============================")
        print(f"📅 Horário da Turma {turma.upper()}")
        print(f"==============================")
        
        tabela = {dia: {h: "" for h in horas} for dia in range(1,6)}

        for uc in ucs:
            for i in [1,2]:
                var = f"{uc}_{i}"
                slot, room, online = solution[var]
                dia = (slot-1)//4 + 1
                hora = horas[(slot-1)%4]
                modo = "ON" if online else "PR"
                tabela[dia][hora] = f"{uc} ({room}, {modo})"

        print(f"{'Hora':<10}  Dia1              Dia2              Dia3              Dia4              Dia5")
        print("-"*95)
        for h in horas:
            linha = f"{h:<10}  "
            for dia in range(1,6):
                cel = tabela[dia][h] if tabela[dia][h] else "-"
                linha += f"{cel:<18} "
            print(linha)
        print("\n")


⏳ Gerando solução...

✅ Solução encontrada!


📅 Horário da Turma T01
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       -                  -                  -                  -                  -                  
11-13       -                  -                  -                  UC13 (R3, PR)      UC11 (Lab01, PR)   
14-16       -                  UC12 (R3, PR)      -                  UC15 (R3, PR)      UC13 (R3, PR)      
16-18       UC11 (Lab01, PR)   UC15 (Lab01, PR)   UC12 (R3, PR)      UC14 (Lab01, PR)   UC14 (Lab01, PR)   



📅 Horário da Turma T02
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       -                  -                  -                  -                  -              

# 3. Agente em execução
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur vitae sapien urna. Ut at aliquet metus, vel porta metus. Curabitur quis tellus at sapien ullamcorper feugiat. Donec auctor, ante at ornare condimentum, magna mi ornare orci, vitae feugiat ligula sapien sed ex. Cras id interdum ipsum. Sed metus nibh, maximus at viverra at, pretium id est. Vivamus semper urna et libero gravida, euismod tincidunt sem hendrerit. Curabitur semper magna eu nisl laoreet congue. Curabitur viverra odio sed neque volutpat, volutpat feugiat nunc finibus. Donec mattis semper odio, eget tincidunt nulla convallis ut. Donec odio diam, facilisis vitae iaculis quis, gravida vel lacus. Sed finibus arcu sit amet dolor lobortis lacinia. Proin vestibulum lectus non lorem tempus aliquam. Cras ex nulla, condimentum sed mauris at, pulvinar finibus erat.

# 4. Conclusão
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.